In [31]:
!pip install opencv-python 

In [12]:
%%cmd
pip install cmake

Microsoft Windows [Version 10.0.19045.2846]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\palsa>pip install cmake
     ---------------------------------------- 33.0/33.0 MB 3.0 MB/s eta 0:00:00

(base) C:\Users\palsa>

In [18]:
%%cmd
python --version

Microsoft Windows [Version 10.0.19045.2846]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\palsa>python --version
Python 3.10.9

(base) C:\Users\palsa>

In [20]:
%%cmd
pip install "C:\Users\palsa\OneDrive\Desktop\Dlib-python whl packages\Dlib-python whl packages\dlib-19.22.99-cp310-cp310-win_amd64.whl"

Microsoft Windows [Version 10.0.19045.2846]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\palsa>pip install "C:\Users\palsa\OneDrive\Desktop\Dlib-python whl packages\Dlib-python whl packages\dlib-19.22.99-cp310-cp310-win_amd64.whl"
Processing c:\users\palsa\onedrive\desktop\dlib-python whl packages\dlib-python whl packages\dlib-19.22.99-cp310-cp310-win_amd64.whl

(base) C:\Users\palsa>

In [26]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25854 sha256=c1e3c58ff9d1feae3f696a17be30c048eb178e9e206faf3be2c1d2341b7c78dd
  Stored in directory: c:\users\palsa\appdata\local\pip\cache\wheels\c2\02\32\f3617a9f68bcc67eda3ebeb4514eba18f62e81ff439428109d
Successfully built imutils


In [1]:
# importing OpenCV Library for basic image processing function
import cv2

In [2]:
#numpy for array related functions
import numpy  as np 

In [3]:
# dlib for deep learning based module AND face landmark detection 
import dlib

In [4]:
from imutils import face_utils

In [5]:
# initializing the camera and taking the instance
cap = cv2.VideoCapture(0)

In [6]:
# initializing the face detector and landmark detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [7]:
#status marking for current state
sleep = 0
drowsy = 0
active = 0
status=""
color=(0,0,0)

In [8]:
def compute(ptA,ptB):
    dist = np.linalg.norm(ptA - ptB)
    return dist

In [9]:
def blinked(a,b,c,d,e,f):
    up = compute(b,d) + compute(c,e)
    down = compute(a,f)
    ratio = up/(2.0*down)
#Checking if it is blinked
    if(ratio>0.25):
        return 2
    elif(ratio>0.21 and ratio<=0.25):
        return 1
    else:
        return 0 

In [ ]:
while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    face_frame = frame.copy()
    #detected face in faces array
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        #The numbers are actually the landmarks which will show eye
        left_blink = blinked(landmarks[36],landmarks[37], 
        landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42],landmarks[43], 
        landmarks[44], landmarks[47], landmarks[46], landmarks[45])
        
        #Now judge what to do for the eye blinks
        if(left_blink==0 or right_blink==0):
            sleep+= 1
            drowsy=0
            active=0
        if(sleep>6):
            status="SLEEPING !!!"
            color = (255,0,0)

        elif(left_blink==1 or right_blink==1):
            sleep=0
            active=0
            drowsy+=1
        if(drowsy>6):
            status="Drowsy !"
            color = (0,0,255)

        else:
            drowsy=0
            sleep=0
            active+=1
        if(active>6):
            status="Active :)"
            color = (0,255,0)
        
        cv2.putText(frame, status, (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color,3)

        for n in range(0, 68):
            (x,y) = landmarks[n]
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)
    cv2.imshow("Frame", frame)
    cv2.imshow("Result of detector", face_frame)
    key = cv2.waitKey(1)
if key == 27:
    break  